In [ ]:
# load all libraries
import os
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from natsort import natsorted
from collections import defaultdict
import cv2
from scipy.stats import linregress
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# Set location of calibration data.
Cali_Drive ="C:/"
Cali_Folder = "YOUR FOLDER"
Cali_Mearsurements = "YOUR SUB-FOLDER"

# Set the location of the measurement images.
Drive ="C:/"
Folder = "YOUR FOLDER"
Mearsurements = "YOUR SUB-FOLDER"

- Load optode calibration parameters dataframe

In [ ]:
#  Laod excel data file and Set Oxygen value to Float
excel_file_path = f'{Cali_Drive}/{Cali_Folder}/{Cali_Mearsurements}/SV_Calibration_Parameters.xlsx'
data = pd.read_excel(excel_file_path)

- Make a linear regression on calibration parameters for Ksv, f and R0 over temperature to function as temperature compensation
- Display linear regressions

In [ ]:
# Initialize linear regression models
global_models = {
    'RO': LinearRegression(),
    'k': LinearRegression(),
    'f': LinearRegression()
}

# Perform global linear regressions and plot setup
plt.figure(figsize=(18, 6))
predictions = {}
fitting_functions = {}

# Perform and plot global regression for each parameter
for i, param in enumerate(['RO', 'k', 'f']):
    y = data[param]
    global_models[param].fit(X, y)
    predicted = global_models[param].predict(X)
    predictions[param] = predicted
    r2 = r2_score(y, predicted)
    fitting_functions[param] = f'{param} = {global_models[param].coef_[0]:.4f} * Temp + {global_models[param].intercept_:.4f}\nR² = {r2:.3f}'
    
    # Plot
    plt.subplot(1, 3, i+1)
    plt.plot(data['Temp'], y, marker='o', linestyle='', markersize=10, color='black', label=param)
    plt.plot(data['Temp'], predicted, linestyle='-', color='red', label='Regression Line')
    plt.xlabel('Temperature (°C)', fontsize=20)
    plt.ylabel(param, fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.text(0.05, 0.95, fitting_functions[param], transform=plt.gca().transAxes, fontsize=16, verticalalignment='top')

plt.tight_layout()
#plt.savefig("r0_k_f_values.png", dpi=300)
plt.show()

# Initialize storage for group-specific linear regression models
model_storage = {}

# Perform groupwise linear regressions
for index, group in data.groupby('Group Index'):
    models = {}
    X_group = group[['Temp']]
    for param in ['RO', 'k', 'f']:
        y_group = group[param]
        model = LinearRegression()
        model.fit(X_group, y_group)
        models[param] = model
    model_storage[index] = models
    
print(model_storage)


- Load RAW measurement images and set a threshold pixel value

In [ ]:
# Set the path to the input directory
input_dir = f'{Drive}/{Folder}/{Mearsurements}/'

# Get a list of all image files in the input directory
input_files = [f for f in os.listdir(input_dir) if f.endswith('.tiff')]
input_files = natsorted(input_files)

# Define the threshold value
threshold_value = 256

# Create a list to store the thresholded images
thresholded_images = []

# Calculate the number of rows and columns for the grid
num_images = len(input_files)
num_rows = int(num_images ** 0.5)
num_cols = (num_images + num_rows - 1) // num_rows

# Create the grid of subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8))

# Iterate over the image files and display them in the grid
for i, filename in enumerate(input_files):
    # Load the image
    image = tifffile.imread(os.path.join(input_dir, filename))

    # Apply the threshold
    thresholded_image = np.where(image > threshold_value, image, 0)
    thresholded_image_nan = np.where(thresholded_image == 0, np.nan, thresholded_image)

    thresholded_images.append(thresholded_image_nan)

    # Determine the subplot indices
    row_idx = i // num_cols
    col_idx = i % num_cols

    # Display the thresholded image in the corresponding subplot
    axes[row_idx, col_idx].imshow(thresholded_image, cmap='viridis', vmin=256,vmax=4065)
    axes[row_idx, col_idx].set_title(filename[15:])
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Display the grid of images
plt.show()


- Save thresholded images in new directory and display one example image to check crop settings.

In [ ]:
# Set output directory
output_dir = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save Thresholded images
for i, filename in enumerate(input_files):
    output_path = os.path.join(output_dir, filename)
    tifffile.imwrite(output_path, thresholded_images[i])
    
# Set the path to the directory containing the thresholded images
thresholded_dir = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/'

# Get a list of all thresholded image files in the directory
thresholded_files = [f for f in os.listdir(thresholded_dir) if f.endswith('.tiff')]
thresholded_files = natsorted(thresholded_files)

test_thresholded_image = tifffile.imread(os.path.join(thresholded_dir,thresholded_files[0]))
# Fullsize crop setting[0:3040,0:4056] 
test_thresholded_image = test_thresholded_image[0:3040,0:4056] 
plt.imshow(test_thresholded_image)

- Create and save image stacks of each color channel.
- Stack averages are made for replicates. 

In [ ]:
input_dir = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/'
output_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Stacks'

if not os.path.exists(output_path):
    os.makedirs(output_path)

input_files = [f for f in os.listdir(input_dir) if f.endswith('.tiff')]
input_files = natsorted(input_files)

# Group files by the identifying number
groups = defaultdict(list)
for file in input_files:
    numeric_num = float(os.path.splitext(file)[0][24:].replace(",", "."))
    groups[numeric_num].append(file)

# Initialize lists to store the averaged channel stacks
averaged_red_stack = []
averaged_green_stack = []
averaged_blue_stack = []

for numeric_num, files in groups.items():
    red_accumulator = []
    green_accumulator = []
    blue_accumulator = []

    # Load and accumulate each replicate image
    for file in files:
        img = tifffile.imread(os.path.join(input_dir, file))

         ### Set Crop dimension (OPTIONAL) ### No crop setting: [0:3040,0:4056]
        img = img[0:3040,0:4056]  
        
        red_channel = img[1::2, 1::2]
        blue_channel = img[0::2, 0::2]
        green_channel_1 = img[0::2, 1::2]
        green_channel_2 = img[1::2, 0::2]
        green_channel = np.add(green_channel_1, green_channel_2) // 2

        red_accumulator.append(red_channel)
        blue_accumulator.append(blue_channel)
        green_accumulator.append(green_channel)
    
    # Calculate the average of replicates for each channel
    red_avg = np.mean(red_accumulator, axis=0)
    green_avg = np.mean(green_accumulator, axis=0)
    blue_avg = np.mean(blue_accumulator, axis=0)
    
    # Add the averaged channels to the corresponding stacks
    averaged_red_stack.append(red_avg)
    averaged_green_stack.append(green_avg)
    averaged_blue_stack.append(blue_avg)

# Stack the averaged channels along the third axis
averaged_red_stack = np.stack(averaged_red_stack, axis=0)
averaged_green_stack = np.stack(averaged_green_stack, axis=0)
averaged_blue_stack = np.stack(averaged_blue_stack, axis=0)

# Save the averaged stacks to separate TIFF files
tifffile.imwrite(os.path.join(output_path, "averaged_stacked_red_channel.tiff"), averaged_red_stack)
tifffile.imwrite(os.path.join(output_path, "averaged_stacked_green_channel.tiff"), averaged_green_stack)
tifffile.imwrite(os.path.join(output_path, "averaged_stacked_blue_channel.tiff"), averaged_blue_stack)

- Load image stacks and get Red/Green intensity ratio and save ratio images in new directory

In [ ]:
output_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Stacks'
# Get a list of all TIFF files in the directory
files = [f for f in os.listdir(output_path) if f.endswith('.tiff')]
print(files)

green_stack = tifffile.imread(os.path.join(output_path, files[1]))
red_stack = tifffile.imread(os.path.join(output_path, files[2]))
blue_stack = tifffile.imread(os.path.join(output_path, files[0]))

#Get length of image stacks
num_images_green = len(green_stack)
num_images_red = len(red_stack)
num_images_blue = len(blue_stack)
if num_images_green == num_images_red and num_images_blue:
    print("Equal stacks")    
else:
    print("unequal stacks")

num_images = num_images_green
num_columns = green_stack[0].shape[1]# Define the number of columns per image

print("number of images in stack:",num_images)
print("number of columns:", num_columns)

output_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Ratio_Images/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

mean_red_intensity =[]
mean_green_intensity = []
mean_blue_intensity = []
std_red_intensity = []
std_green_intensity = []
std_blue_intensity =[]

# Initialize lists to store column-wise means and stds for the red/green ratio
mean_red_green_ratio_per_column = []
std_red_green_ratio_per_column = []

for i in range(num_images):
    mean_red_intensity.append(np.mean(red_stack[i]))
    mean_green_intensity.append(np.mean(green_stack[i]))
    mean_blue_intensity.append(np.mean(blue_stack[i]))
    std_red_intensity.append(np.std(red_stack[i]))
    std_green_intensity.append(np.std(green_stack[i]))
    std_blue_intensity.append(np.std(blue_stack[i]))

    # Calculate the intensity ratio for each pixel
    ratio = red_stack[i] / green_stack[i]

    # Construct the output file path
    output_file = os.path.join(output_path, f"ratio_image_{i}.tiff")
    # Save the new image
    tifffile.imwrite(output_file, ratio)

    # Calculate the mean and std of the ratio for each column
    mean_ratio_per_column = np.mean(ratio, axis=0)  # Average across rows for each column
    std_ratio_per_column = np.std(ratio, axis=0)    # Standard deviation across rows for each column

    mean_red_green_ratio_per_column.append(mean_ratio_per_column)
    std_red_green_ratio_per_column.append(std_ratio_per_column)

# Output the column-wise means for the first image as an example
print(mean_red_green_ratio_per_column[0])

- Get Ratio images and apply modified Stern-Volmer fir using the predicted Ksv, f and RO for the set temperature.
- Remember to adjust group_index value of groups were used in calibration
- Remember to adjust temp to adjust for temperature during measurement. 

In [ ]:
# Set the path to the directory containing the ratio images
ratio_img_dir = f'{Drive}/{Folder}/{Mearsurements}/Ratio_Images/'
output_path = f'{Drive}/{Folder}/{Mearsurements}/Ratio_Images_Calibrated/'

# Set Group index to match image coulmn groups used in the optode calibration.
group_index = 30

# Determine the temperature (Celsius) for the current images 
temp = 22 

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Get a list of all TIFF files in the input directory and sort them
files = natsorted([f for f in os.listdir(ratio_img_dir) if f.endswith('.tiff')])

# Function to calculate oxygen concentration
def calculate_x(y, f, k):
    x = (f / (1 / y - (1 - f)) - 1) / k
    return x

# Calculate and save the images
for i, file in enumerate(files):

    ratio_img = tifffile.imread(os.path.join(ratio_img_dir, file))

    # Initialize an array for calibrated O2 concentration
    O2_concentration = np.zeros_like(ratio_img, dtype=np.float32)

    # Iterate over column indices to calculate O2 concentration
    for col_index in range(ratio_img.shape[1]):
        group_index =col_index//group_index
        if group_index in model_storage:
            models = model_storage[group_index]
            f_pred = models['f'].predict([[temp]])[0]
            k_pred = models['k'].predict([[temp]])[0]
            R0_pred = models['R0'].predict([[temp]])[0]

            # Calibrate the image for this column
            R0_over_R = R0_pred / ratio_img[:, col_index]
            O2_concentration[:, col_index] = calculate_x(R0_over_R, f_pred, k_pred)

    # Save the calibrated image
    output_file = os.path.join(output_path, f"calibrated_{file}")
    tifffile.imwrite(output_file, O2_concentration)

print("Processing complete.")


- Get the calibrated images and display falsecolor images.

In [ ]:
# Set the path to the directory containing the calibrated images
calibrated_img_dir = f'{Drive}/{Folder}/{Mearsurements}/Ratio_Images_Calibrated/'

# Get a list of all TIFF files in the calibrated images directory and sort them naturally
files = natsorted([f for f in os.listdir(calibrated_img_dir) if f.endswith('.tiff')])


# Display the normalized images using matplotlib
for i, file in enumerate(files):
    calibrated_image = tifffile.imread(os.path.join(calibrated_img_dir, file))

    # Flip the image by 180 degrees using numpy
    #flipped_image = np.flipud(np.fliplr(calibrated_image))

    plt.imshow(calibrated_image, cmap='inferno',vmin=0,vmax=100)
    plt.title(f"Calibrated Image {i}")
    plt.colorbar(label='O2 Concentration (%)')
    plt.show()

Save falsecolor images as 8bit images (needed for image stitching post-processing)

In [ ]:
# Set the path to the directory containing the calibrated images
calibrated_img_dir = f'{Drive}/{Folder}/{Mearsurements}/Ratio_Images_Calibrated/'

# Get a list of all TIFF files in the calibrated images directory and sort them naturally
files = natsorted([f for f in os.listdir(calibrated_img_dir) if f.endswith('.tiff')])

# Directory to save the images with the specified colormap range as TIFF files
output_dir = f'{Folder}/{Mearsurements}/Images_Calibrated_FalseColor/'
os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

# Define the desired colormap range
colormap_min = 0 # Set minimum value
colormap_max = 100 # Set maximum value

for i, file in enumerate(files):
    calibrated_image = tifffile.imread(os.path.join(calibrated_img_dir, file))

    # Apply the colormap with the specified range
    cmap = plt.get_cmap('inferno')
    normed_image = cmap((calibrated_image - colormap_min) / (colormap_max - colormap_min))

    # Convert the image to uint8 (8-bit) data type
    normed_image_uint8 = (normed_image * 255.0).astype('uint8')

    # Save the image as a TIFF file with the specified colormap range and uint8 data type
    output_filename = os.path.join(output_dir, f"calibrated_image_{i}.tiff")
    tifffile.imwrite(output_filename, normed_image_uint8)

print("Images saved as TIFF with uint8 data type to:", output_dir)